In [1]:
from tmrl import get_environment
from time import sleep
from math import floor, sqrt
#from tmrl.custom.custom_models import conv2d_out_dims, num_flat_features, mlp
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import tmrl.config.config_constants as cfg
from collections import deque, namedtuple

import random


Hyperparameters and Device

In [2]:
BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 64         # minibatch size
GAMMA = 0.99            # discount factor
TAU = 1e-3              # for soft update of target parameters
LR = 5e-4               # learning rate 
UPDATE_EVERY = 4        # how often to update the network
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


CNN Model for Training

In [3]:
# Model paramters
image_height = 64  #height of images
image_width = 64   #width of images
num_images = 4  #number of images (tmrl returns 4 to us)
keep_prob = 0.5     

In [62]:
class CNN_Trackmania_Critic(nn.Module):
    def __init__(self, seed):
        super(CNN_Trackmania_Critic, self).__init__()

        self.IMAGE_HEIGHT = image_height
        self.IMAGE_WIDTH = image_width
        self.keep_prob = keep_prob
        self.seed = torch.manual_seed(seed)

        # convolutional layers
        self.conv1 = nn.Conv2d(num_images, 24, kernel_size=8, stride=2)
        self.conv2 = nn.Conv2d(24, 36, kernel_size=4, stride=2)
        self.conv3 = nn.Conv2d(36, 48, kernel_size=4, stride=2)
        self.conv4 = nn.Conv2d(48, 64, kernel_size=4, stride=2)

        # dropout layer
        self.dropout = nn.Dropout(keep_prob)

        # fully-connected layers
        self.fc1 = nn.Linear(64 * 1 * 1 + 9, 100)
        self.fc2 = nn.Linear(100, 50)
        self.fc3 = nn.Linear(50, 10)
        self.fc4 = nn.Linear(10, 3)

    def forward(self, x):
        speed, gear, rpm, images, act1, act2 = x
        images = torch.from_numpy(images).float().to(device)
        speed = torch.from_numpy(speed).to(device)
        gear = torch.from_numpy(gear).to(device)
        rpm = torch.from_numpy(rpm).to(device)
        act1 = torch.from_numpy(act1).to(device)
        act2 = torch.from_numpy(act2).to(device)

        x = F.tanh(self.conv1(images))
        x = F.tanh(self.conv2(x))
        x = F.tanh(self.conv3(x))
        x = F.tanh(self.conv4(x))

        x = self.dropout(x)

        x = x.view(-1)
        act1 = act1.view(-1)
        act2 = act2.view(-1)
        #print(x.size())
        x = torch.cat((speed, gear, rpm, x, act1, act2), -1)
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = F.tanh(self.fc3(x))
        x = self.fc4(x)

        return x

In [ ]:
class CNN_Trackmania_Critic(nn.Module):
    def __init__(self, seed):
        super(CNN_Trackmania_Critic, self).__init__()

        self.IMAGE_HEIGHT = image_height
        self.IMAGE_WIDTH = image_width
        self.keep_prob = keep_prob
        self.seed = torch.manual_seed(seed)

        # convolutional layers
        self.conv1 = nn.Conv2d(num_images, 24, kernel_size=8, stride=2)
        self.conv2 = nn.Conv2d(24, 36, kernel_size=4, stride=2)
        self.conv3 = nn.Conv2d(36, 48, kernel_size=4, stride=2)
        self.conv4 = nn.Conv2d(48, 64, kernel_size=4, stride=2)

        # dropout layer
        self.dropout = nn.Dropout(keep_prob)

        # fully-connected layers
        self.fc1 = nn.Linear(64 * 1 * 1 + 9, 100)
        self.fc2 = nn.Linear(100, 50)
        self.fc3 = nn.Linear(50, 10)
        self.fc4 = nn.Linear(10, 3)

    def forward(self, x):
        speed, gear, rpm, images, act1, act2 = x
        images = torch.from_numpy(images).float().to(device)
        speed = torch.from_numpy(speed).to(device)
        gear = torch.from_numpy(gear).to(device)
        rpm = torch.from_numpy(rpm).to(device)
        act1 = torch.from_numpy(act1).to(device)
        act2 = torch.from_numpy(act2).to(device)

        x = F.tanh(self.conv1(images))
        x = F.tanh(self.conv2(x))
        x = F.tanh(self.conv3(x))
        x = F.tanh(self.conv4(x))

        x = self.dropout(x)

        x = x.view(-1)
        act1 = act1.view(-1)
        act2 = act2.view(-1)
        #print(x.size())
        x = torch.cat((speed, gear, rpm, x, act1, act2), -1)
        x = F.tanh(self.fc1(x))
        x = F.tanh(self.fc2(x))
        x = F.tanh(self.fc3(x))
        x = self.fc4(x)

        return x

In [60]:
class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, seed):
        """Initialize a ReplayBuffer object.

        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            batch_size (int): size of each training batch
            seed (int): random seed
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)  
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
        self.seed = random.seed(seed)
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)

        states = np.vstack([e.state for e in experiences if e is not None])
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = np.vstack([e.next_state for e in experiences if e is not None])
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
  
        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

Agent - Used from HW7 for a DeepQNetwork

In [78]:
class CNNAgent():
    """Interacts with and learns from the environment."""

    def __init__(self, state_size, action_size, seed):
        """Initialize an Agent object.
        
        Params
        ======
            state_size (int): dimension of each state
            action_size (int): dimension of each action
            seed (int): random seed
        """
        self.state_size = state_size
        self.action_size = action_size
        self.seed = random.seed(seed)

        # Q-Network
        self.qnetwork_local = CNN_Trackmania(seed).to(device)
        self.qnetwork_target = CNN_Trackmania(seed).to(device)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=LR)

        # Replay memory
        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE, seed)
        # Initialize time step (for updating every UPDATE_EVERY steps)
        self.t_step = 0
    
    def step(self, state, action, reward, next_state, done):
        # Save experience in replay memory
        """ speed, gear, rpm, images, act1, act2 = state
        images = torch.from_numpy(images).float().to(device)
        speed = torch.from_numpy(speed).to(device)
        gear = torch.from_numpy(gear).to(device)
        rpm = torch.from_numpy(rpm).to(device)
        act1 = torch.from_numpy(act1).to(device)
        act2 = torch.from_numpy(act2).to(device)

        next_speed, next_gear, next_rpm, next_images, next_act1, next_act2 = next_state
        next_images = torch.from_numpy(next_images).float().to(device)
        next_speed = torch.from_numpy(next_speed).to(device)
        next_gear = torch.from_numpy(next_gear).to(device)
        next_rpm = torch.from_numpy(next_rpm).to(device)
        next_act1 = torch.from_numpy(next_act1).to(device)
        next_act2 = torch.from_numpy(next_act2).to(device)

        state = speed, gear, rpm, images, act1, act2
        next_state = next_speed, next_gear, next_rpm, next_images, next_act1, next_act2  """
        self.memory.add(state, action, reward, next_state, done)
        
        '''stop coding here'''
        # Learn every UPDATE_EVERY time steps.
        self.t_step = (self.t_step + 1) % UPDATE_EVERY
        if self.t_step == 0:
            # If enough samples are available in memory, get random subset and learn
            if len(self.memory) > BATCH_SIZE:
                experiences = self.memory.sample()
                self.learn(experiences, GAMMA)

    def act(self, state, eps=0.):
        """Returns actions for given state as per current policy.
        
        Params
        ======
            state (array_like): current state
            eps (float): epsilon, for epsilon-greedy action selection
        """
        #state = torch.from_numpy(state).unsqueeze(0).to(device)
        self.qnetwork_local.eval()
        with torch.no_grad():
            action_values = self.qnetwork_local.forward(state)
        self.qnetwork_local.train()

        # Epsilon-greedy action selection
        if random.random() > eps:
            return action_values.cpu().data.numpy()
        else:
            return [random.uniform(-1,1),random.uniform(-1,1),random.uniform(-1,1)]

    def learn(self, experiences, gamma):
        """Update value parameters using given batch of experience tuples.

        Params
        ======
            experiences (Tuple[torch.Variable]): tuple of (s, a, r, s', done) tuples 
            gamma (float): discount factor
        """
        # Obtain random minibatch of tuples from D
        states, actions, rewards, next_states, dones = experiences

        #print(next_states)
        
        ## Compute and minimize the loss
        ### Extract next maximum estimated value from target network
        print(next_states.shape)
        max_q_targets_next = 0
        for next_state in next_states:
            print(next_state)
            q_targets_next = self.qnetwork_target(next_state).detach().unsqueeze(1)
            if q_targets_next > 
        
        '''Your code here'''
        ### Calculate target value from bellman equation
        ### Make sure to include a dones calculation, where q_targets is left alone if dones is 0 
        ### and is just the reward if dones is 1
        
        #if dones = 1, q_targets = 1. If dones is 0,
        q_targets = rewards + (gamma*q_targets_next)*(1-dones)
        
        '''Stop coding here'''
        ### Calculate expected value from local network
        q_expected = self.qnetwork_local(states).gather(1, actions)
        
        ### Loss calculation (we used Mean squared error)
        loss = F.mse_loss(q_expected, q_targets)
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()

        # ------------------- update target network ------------------- #
        self.soft_update(self.qnetwork_local, self.qnetwork_target, TAU)                     

    def soft_update(self, local_model, target_model, tau):
        """Soft update model parameters.
        θ_target = τ*θ_local + (1 - τ)*θ_target

        Params
        ======
            local_model (PyTorch model): weights will be copied from
            target_model (PyTorch model): weights will be copied to
            tau (float): interpolation parameter 
        """
        for target_param, local_param in zip(target_model.parameters(), local_model.parameters()):
            target_param.data.copy_(tau*local_param.data + (1.0-tau)*target_param.data)

In [79]:

#model = CNN_Trackmania(image_height, image_width, num_images, keep_prob)
# Let us retrieve the TMRL Gymnasium environment.
# The environment you get from get_environment() depends on the content of config.json
env = get_environment()
agent = CNNAgent(state_size=6,action_size=3, seed=0) #State: speed, gear, rpm, x (4 images), act1, act2
                                                     #Action: speed, gear, rpm between -1 and 1
sleep(3)  # just so we have time to focus the TM20 window after starting the script


eps_start = 1
eps_end=0.01
eps_decay=0.995
scores = []
scores_window = deque(maxlen=100)  # last 100 scores
eps = eps_start
for i_episode in range(1, 2000):
    score = 0 #initialize episode score to 0
    obs, info = env.reset() #get state when lunar lander is restarted
    for _ in range(20000):  # rtgym ensures this runs at 20Hz by default
        #act = model(torch.from_numpy(obs[3]))  # compute action
        #print(obs)
        #print(obs)
        #obs = np.asarray(obs).astype(np.float32)
        act = agent.act(obs, eps)
        #act = act.detach().numpy()
        
        act = np.array(act)
        #print(act)
        next_obs, rew, terminated, truncated, info = env.step(act)  # step (rtgym ensures healthy time-steps)
        agent.step(obs, act, rew, next_obs, terminated) #step agent and learn reward from given action
        obs = next_obs
        score += rew
        if terminated or truncated:
            break
    scores_window.append(score)       # save most recent score
    scores.append(score)              # save most recent score
    eps = max(eps_end, eps_decay*eps) # decrease epsilon
    if np.mean(scores_window)>=200.0:
            print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
            torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
    print("Episode " + str(i_episode) + ": " + str(score))
env.wait()  # rtgym-specific method to artificially 'pause' the environment when needed

c:\Users\Chris\AppData\Local\Programs\Python\Python37\lib\site-packages\torch\nn\functional.py:1956: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


[array([0.54322565], dtype=float32) array([1.], dtype=float32)
 array([186.06776], dtype=float32)
 array([[[143, 142, 141, ..., 141, 141, 141],
         [143, 143, 143, ..., 141, 142, 143],
         [145, 144, 144, ..., 142, 143, 143],
         ...,
         [216, 216, 216, ..., 218, 217, 217],
         [216, 216, 217, ..., 217, 218, 217],
         [216, 217, 216, ..., 217, 217, 218]],

        [[143, 142, 141, ..., 141, 141, 141],
         [143, 143, 143, ..., 141, 142, 143],
         [145, 144, 144, ..., 142, 143, 143],
         ...,
         [216, 216, 216, ..., 218, 217, 217],
         [216, 216, 217, ..., 217, 218, 217],
         [216, 217, 216, ..., 217, 218, 218]],

        [[143, 142, 141, ..., 141, 141, 141],
         [143, 143, 143, ..., 141, 142, 143],
         [145, 144, 144, ..., 142, 143, 143],
         ...,
         [216, 216, 216, ..., 218, 217, 218],
         [216, 216, 217, ..., 217, 217, 217],
         [216, 217, 216, ..., 217, 218, 218]],

        [[143, 142, 141, .

c:\Users\Chris\AppData\Local\Programs\Python\Python37\lib\site-packages\numpy\core\shape_base.py:121: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  ary = asanyarray(ary)


ValueError: too many values to unpack (expected 6)

In [ ]:
print('State shape: ', env.observation_space.shape)
print('Number of actions: ', env.action_space)
